<a href="https://colab.research.google.com/github/anupj/fine-tuning-llama/blob/main/finetune_llama_3_together_ai_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning and Inference using Low-Rank Adaptations(LoRA)


In this notebook I will:

- perform LoRA fine-tuning on Together AI
- perform LoRA inference on the trained model
- swap and perform inference using various LoRA fine-tunes!

In [19]:
!pip install -qU together
os.environ["UV_SYSTEM_PYTHON"] = "1"
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 53.0 MB/s eta 0:00:00


In [21]:
from together import Together
import os
try:
    from google.colab import userdata
    os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')
    os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
except ImportError:
    print("Not in Google Colab environment")

for key in ['TOGETHER_API_KEY', 'WANDB_API_KEY']:
    try:
        api_key = os.environ[key]
        if not api_key:
            raise ValueError(f"{key} environment variable is empty")
    except KeyError:
        api_key = input(f"{key} environment variable is not set. Please enter your API key: ")
        os.environ[key] = api_key

client = Together(api_key = TOGETHER_API_KEY)


## Perform LoRA Fine-tune

In [22]:
# Upload dataset to Together AI

train_file_resp = client.files.upload("datasets/small_coqa_10.jsonl", check=True)

print(train_file_resp.id)

Uploading file small_coqa_10.jsonl: 100%|██████████| 33.4k/33.4k [00:00<00:00, 63.2kB/s]


file-cc6d531d-190b-4d3d-a26f-94193aa9362e


**Let's fine tune `Llama-3.2-1B-Instruct` model**

In [27]:
ft_resp = client.fine_tuning.create(
    training_file   = train_file_resp.id,
    model           = 'meta-llama/Llama-3.2-1B-Instruct',
    train_on_inputs = "auto",
    n_epochs        = 3,
    n_checkpoints   = 1,
    wandb_api_key   = WANDB_API_KEY,
    lora            = True,
    warmup_ratio    = 0,
    learning_rate   = 1e-5,
    suffix          = 'FT-webinar-demo-1b',
)

print(ft_resp.id)

ft-5bf7b814


In [32]:
print(ft_resp.output_name)
print(ft_resp.id)

anupjadhav/Llama-3.2-1B-Instruct-FT-webinar-demo-1b-7967b7fd
ft-5bf7b814


## LoRA Inference


In [35]:
model_name = ft_resp.output_name
user_prompt = "what did Venters call Lassiter?"

response = client.chat.completions.create(
  model = model_name + '-adapter',
  messages = [
      {
          "role": "user",
          "content": user_prompt,
      }
  ],
  max_tokens = 224,
  temperature = 0.7,
)

print(response.choices[0].message.content)

In the movie 007 James Bond, Bond's partner is Lassiter and character 007 James Bond also has a nickname for Lassiter, which is 'The Ginger Nuts of Chemical Weapons'.
